# Graph Data structures

Topological ordering

In [ ]:
MATCH (activity:ACTIVITY)
OPTIONAL MATCH (activity)<-[:DEPENDS_ON]-(predecessor:ACTIVITY)
WITH activity, COUNT(predecessor) AS incomingDependencies
OPTIONAL MATCH path = (activity)-[:DEPENDS_ON*]->(dependent:ACTIVITY)
WITH activity, incomingDependencies, COALESCE(MAX(LENGTH(path)), 0) AS depth
RETURN 
    activity.name AS activityName, 
    activity.description AS description, 
    activity.duration AS duration, 
    depth, 
    incomingDependencies
ORDER BY depth DESC, incomingDependencies ASC


Early start and early finish topological ordering

In [ ]:
MATCH (activity:ACTIVITY)
OPTIONAL MATCH (activity)<-[:DEPENDS_ON]-(predecessor:ACTIVITY)
WITH activity, COLLECT(predecessor) AS predecessors
ORDER BY activity.depth ASC
WITH activity,
    REDUCE(maxFinish = 0, pred IN predecessors | 
        CASE 
            WHEN pred.earlyFinish IS NOT NULL AND pred.earlyFinish > maxFinish THEN pred.earlyFinish 
            ELSE maxFinish 
        END) AS earlyStart
SET activity.earlyStart = COALESCE(earlyStart, 0),
    activity.earlyFinish = COALESCE(earlyStart, 0) + activity.duration
RETURN activity.name AS activityName, activity.earlyStart, activity.earlyFinish
ORDER BY activity.depth ASC

In [1]:
"""
MATCH (activity:ACTIVITY)
OPTIONAL MATCH (activity)<-[:DEPENDS_ON]-(predecessor:ACTIVITY)
WITH activity, COUNT(predecessor) AS incomingDependencies
OPTIONAL MATCH path = (activity)-[:DEPENDS_ON*]->(dependent:ACTIVITY)
WITH activity, incomingDependencies, COALESCE(MAX(LENGTH(path)), 0) AS depth
RETURN 
    activity.name AS activityName, 
    activity.description AS description, 
    activity.duration AS duration, 
    depth, 
    incomingDependencies
ORDER BY depth DESC, incomingDependencies ASC

MATCH (activity:ACTIVITY)
OPTIONAL MATCH (activity)<-[:DEPENDS_ON]-(predecessor:ACTIVITY)
WITH activity, COLLECT(predecessor) AS predecessors
ORDER BY activity.depth ASC
WITH activity,
    REDUCE(maxFinish = 0, pred IN predecessors | 
        CASE 
            WHEN pred.earlyFinish IS NOT NULL AND pred.earlyFinish > maxFinish THEN pred.earlyFinish 
            ELSE maxFinish 
        END) AS earlyStart
SET activity.earlyStart = COALESCE(earlyStart, 0),
    activity.earlyFinish = COALESCE(earlyStart, 0) + activity.duration
RETURN activity.name AS activityName, activity.earlyStart, activity.earlyFinish
ORDER BY activity.depth ASC
"""

SyntaxError: invalid syntax (175008303.py, line 1)

In [ ]:
"""
OPTIONAL MATCH (activity)<-[:DEPENDS_ON]-(predecessor:ACTIVITY)    
WITH activity, predecessor, COUNT(predecessor) AS incomingDependencies     

OPTIONAL MATCH path = (activity)-[:DEPENDS_ON*]->(dependent:ACTIVITY) 
WITH activity, predecessor, incomingDependencies, COALESCE(MAX(LENGTH(path)), 0) AS depth 
ORDER BY depth ASC, incomingDependencies ASC   // activity with longest chain comes first
WITH activity, COLLECT(predecessor) AS predecessors                 // group all predecessor nodes into a list
WITH activity,
    REDUCE(maxFinish = 0, pred IN predecessors |                    //  Iterate over each predecessor for forward pass take max of predecessors early finish 
        CASE 
            WHEN pred.earlyFinish IS NOT NULL AND pred.earlyFinish > maxFinish THEN pred.earlyFinish  // get max early finish of predecessors
            ELSE maxFinish 
        END) AS earlyStart                                          // the maxfinish in the predecessors will be the early start of current node  or 0 if no predecessors acc maxFinish = 0                     
SET activity += {earlyStart: COALESCE(earlyStart, 0),
    earlyFinish: COALESCE(earlyStart, 0) + activity.duration} 
RETURN activity.name AS activityName, activity.earlyStart, activity.earlyFinish
ORDER BY activity.depth ASC;"""

In [ ]:
"""
// initialize earlystart and early finish of the last node
MATCH (activity:ACTIVITY)
WHERE NOT (activity)-[:DEPENDS_ON]->()  // Find node with no outgoing relationship
SET activity.lateFinish = activity.earlyFinish,
    activity.lateStart = activity.lateFinish - activity.duration;

// Propagate lateStart and lateFinish values backward
MATCH (activity:ACTIVITY)
OPTIONAL MATCH (activity)-[:DEPENDS_ON]->(successor:ACTIVITY) // Match successors
WITH activity, 
     COLLECT(successor) AS successors                         // Group successors for each node
ORDER BY activity.depth DESC                                  // Process nodes in reverse order of depth
WITH activity,
     REDUCE(minStart = activity.lateFinish, succ IN successors | 
         CASE 
             WHEN succ.lateStart IS NOT NULL AND succ.lateStart < minStart THEN succ.lateStart
             ELSE minStart
         END) AS lateFinish                                 // Calculate lateFinish as min(lateStart) of successors
SET activity.lateFinish = COALESCE(lateFinish, activity.earlyFinish),
    activity.lateStart = activity.lateFinish - activity.duration;

// Step 3: Return Results
MATCH (activity:ACTIVITY)
RETURN activity.name AS activityName, activity.lateStart, activity.lateFinish
ORDER BY activity.depth DESC;
"""

## topological order
Only works for Directed Acyclic Graphs

In [ ]:
"""// Match all activities and their predecessors
MATCH (activity:ACTIVITY)
OPTIONAL MATCH (activity)<-[:DEPENDS_ON]-(predecessor:ACTIVITY)

// Collect predecessors for each activity
WITH activity, COLLECT(predecessor.name) AS predecessors

// Find the depth of each activity for topological order
OPTIONAL MATCH path = (activity)-[:DEPENDS_ON*]->(dependent:ACTIVITY)
WITH activity, predecessors, 
     COALESCE(MAX(LENGTH(path)), 0) AS depth, 
     SIZE(predecessors) AS incomingDependencies

// Order activities by depth (descending) and by number of incoming dependencies (ascending)
ORDER BY depth DESC, incomingDependencies ASC 

// Return the results with the predecessors as a list
RETURN 
    activity.name AS name, 
    depth, 
    incomingDependencies, 
    predecessors AS predecessorNames
"""

### Forward pass
It does calculate the correct values but one has to run the query a couple of time for it to calculate the correct values. Reason is Neo4j is propagating stale values for earlyStart and earlyFinish to the next node.

In [ ]:
"""// Match all activities and their predecessors
MATCH (activity:ACTIVITY)
OPTIONAL MATCH (activity)<-[:DEPENDS_ON]-(predecessor:ACTIVITY)

// Collect predecessors for each activity
WITH activity, COLLECT(predecessor) AS predecessors

// Find the depth of each activity for topological order
OPTIONAL MATCH path = (activity)-[:DEPENDS_ON*]->(dependent:ACTIVITY)
WITH activity, predecessors, 
     COALESCE(MAX(LENGTH(path)), 0) AS depth, 
     SIZE(predecessors) AS incomingDependencies

// Order activities by depth (descending) and by number of incoming dependencies (ascending) topological order
ORDER BY depth DESC, incomingDependencies ASC 

// Calculate earlyStart and earlyFinish for each activity
WITH COLLECT([activity, predecessors]) AS activityPredecessorsList
UNWIND activityPredecessorsList AS activityPredecessorsTuple
WITH activityPredecessorsTuple[0] AS currentActivity, 
     REDUCE(
         maxFinish = 0, 
         pred IN activityPredecessorsTuple[1] |                    
         CASE 
             WHEN pred IS NOT NULL AND pred.earlyFinish IS NOT NULL AND pred.earlyFinish > maxFinish 
             THEN pred.earlyFinish  
             ELSE maxFinish 
         END
     ) AS earlyStart
SET currentActivity += {
    earlyStart: COALESCE(earlyStart, 0), 
    earlyFinish: COALESCE(earlyStart, 0) + currentActivity.duration
}
RETURN currentActivity.name AS activityName, 
       currentActivity.earlyStart AS earlyStart, 
       currentActivity.earlyFinish AS earlyFinish
ORDER BY currentActivity.depth ASC
"""

### backward pass

In [ ]:
"""
// match all activities and their successors
MATCH (activity:ACTIVITY)
OPTIONAL MATCH (activity)-[:DEPENDS_ON]->(successor:ACTIVITY)

// collect successors for each activity
WITH activity, COLLECT(successor) AS successors

// find the depth of each activity for topological order
OPTIONAL MATCH path = (activity)<-[:DEPENDS_ON*]-(dependent:ACTIVITY)
WITH activity, successors, 
    COALESCE(MAX(LENGTH(path)), 0) AS depth, 
    SIZE(successors) AS outgoingDependencies

// order activities by depth (descending) and by number of outgoing dependencies (ascending) topological order
ORDER BY depth DESC, outgoingDependencies ASC 

// calculate lateStart and lateFinish for each activity
WITH COLLECT([activity, successors]) AS activitySuccessorsList
UNWIND activitySuccessorsList AS activitySuccessorsTuple
WITH activitySuccessorsTuple[0] AS currentActivity, activitySuccessorsTuple
    WITH currentActivity, activitySuccessorsTuple, REDUCE(
        minStart = currentActivity.earlyFinish, 
        succ IN activitySuccessorsTuple[1] |                    
        CASE 
            WHEN succ IS NOT NULL AND succ.lateStart IS NOT NULL AND succ.lateStart < minStart 
            THEN succ.lateStart  
            ELSE minStart 
        END
    ) AS lateFinish

// update lateStart and lateFinish for each activity
SET currentActivity += {
    lateFinish: COALESCE(lateFinish, currentActivity.earlyFinish), 
    lateStart: COALESCE(lateFinish, currentActivity.earlyFinish) - currentActivity.duration
}

// return the results
RETURN currentActivity.name AS activityName, 
    currentActivity.lateStart AS lateStart, 
    currentActivity.lateFinish AS lateFinish
ORDER BY currentActivity.depth DESC"""

## Calculating Slack
Free slack
Total slack

In [ ]:
"""
MATCH (activity:ACTIVITY)
OPTIONAL MATCH (activity)-[:DEPENDS_ON]->(successor:ACTIVITY)

WITH activity, COLLECT(successor) AS successors

OPTIONAL MATCH path = (activity)<-[:DEPENDS_ON*]-(dependent:ACTIVITY)
WITH activity, successors, 
    COALESCE(MAX(LENGTH(path)), 0) AS depth, 
    SIZE(successors) AS outgoingDependencies

ORDER BY depth DESC, outgoingDependencies ASC 

WITH COLLECT([activity, successors]) AS activitySuccessorsList
UNWIND activitySuccessorsList AS activitySuccessorsTuple
WITH activitySuccessorsTuple[0] AS currentActivity, activitySuccessorsTuple
    WITH currentActivity, activitySuccessorsTuple, REDUCE(
        minSuccEarlyStart = activitySuccessorsTuple[1][0].earlyStart, 
        succ IN activitySuccessorsTuple[1] |                    
        CASE 
            WHEN succ IS NOT NULL AND succ.earlyStart IS NOT NULL AND succ.earlyStart < minSuccEarlyStart 
            THEN succ.earlyStart  
            ELSE minSuccEarlyStart 
        END
    ) AS minSuccEarlyStart
SET currentActivity += {
    totalFloat: currentActivity.lateFinish - currentActivity.earlyFinish,
    freeFloat: COALESCE(minSuccEarlyStart - currentActivity.earlyStart - currentActivity.duration, 0)
}
RETURN currentActivity.name AS activityName, 
    currentActivity.earlyStart AS earlyStart, 
    currentActivity.earlyFinish AS earlyFinish,
    currentActivity.lateStart AS lateStart,
    currentActivity.lateFinish AS lateFinish,
    currentActivity.totalFloat AS totalFloat,
    currentActivity.freeFloat AS freeFloat
ORDER BY currentActivity.depth DESC"""

### Critical path

In [ ]:
"""
// trace paths between critical activities
OPTIONAL MATCH path = (start:ACTIVITY)-[:DEPENDS_ON*]->(end:ACTIVITY)
WHERE ALL(node IN NODES(path) WHERE node.totalFloat = 0)

//Total duration between the critical activities
WITH path, 
     REDUCE(totalDuration = 0, activity IN NODES(path) | totalDuration + activity.duration) AS pathDuration
ORDER BY pathDuration DESC

// Return the longest path and duration
LIMIT 1
RETURN [activity IN NODES(path) | activity.name] AS criticalPath, pathDuration
"""